In [1]:
import pandas as pd
import numpy as np
import scipy.io
import h5py
import mat73

from tqdm import tqdm

from pynwb import NWBHDF5IO

import pyarrow as pa
import pyarrow.parquet as pq

from pyspark.sql import SparkSession
from pyspark.sql import functions as F

In [2]:
import h5py

file = h5py.File('data/000070/sub-Jenkins/sub-Jenkins_ses-20090918_behavior+ecephys.nwb')

io = NWBHDF5IO(file=file, mode='r', load_namespaces=True)

/opt/anaconda3/envs/xfads2/lib/python3.11/site-packages/hdmf/spec/namespace.py:535: UserWarning: Ignoring cached namespace 'hdmf-common' version 1.3.0 because version 1.8.0 is already loaded.
  warn("Ignoring cached namespace '%s' version %s because version %s is already loaded."
/opt/anaconda3/envs/xfads2/lib/python3.11/site-packages/hdmf/spec/namespace.py:535: UserWarning: Ignoring cached namespace 'core' version 2.2.5 because version 2.7.0 is already loaded.
  warn("Ignoring cached namespace '%s' version %s because version %s is already loaded."


In [3]:
nwb = io.read()
nwb

,x,y,z,imp,location,filtering,group,group_name,gain,offset
id,,,,,,,,,,
1,NaN,NaN,NaN,-1.0,"Pre-Motor Cortex, dorsal",1000Hz,"1 pynwb.ecephys.ElectrodeGroup at 0x5088138896\nFields:\n description: array corresponding to device implanted at PMd\n device: Utah Array(PMd) pynwb.device.Device at 0x5087765776\nFields:\n description: 96 channel utah array\n manufacturer: BlackRock Microsystems\n\n location: Caudal, dorsal Pre-motor cortex, Left hemisphere\n",1,1.0,0.0
2,NaN,NaN,NaN,-1.0,"Pre-Motor Cortex, dorsal",1000Hz,"1 pynwb.ecephys.ElectrodeGroup at 0x5088138896\nFields:\n description: array corresponding to device implanted at PMd\n device: Utah Array(PMd) pynwb.device.Device at 0x5087765776\nFields:\n description: 96 channel utah array\n manufacturer: BlackRock Microsystems\n\n location: Caudal, dorsal Pre-motor cortex, Left hemisphere\n",1,1.0,0.0
3,NaN,NaN,NaN,-1.0,"Pre-Motor Cortex, dorsal",1000Hz,"1 pynwb.ecephys.ElectrodeGroup at 0x5088138896\nFields:\n description: array corresponding to device implanted at PMd\n device: Utah Array(PMd) pynwb.device.Device at 0x5087765776\nFields:\n description: 96 channel utah array\n manufacturer: BlackRock Microsystems\n\n location: Caudal, dorsal Pre-motor cortex, Left hemisphere\n",1,1.0,0.0
4,NaN,NaN,NaN,-1.0,"Pre-Motor Cortex, dorsal",1000Hz,"1 pynwb.ecephys.ElectrodeGroup at 0x5088138896\nFields:\n description: array corresponding to device implanted at PMd\n device: Utah Array(PMd) pynwb.device.Device at 0x5087765776\nFields:\n description: 96 channel utah array\n manufacturer: BlackRock Microsystems\n\n location: Caudal, dorsal Pre-motor cortex, Left hemisphere\n",1,1.0,0.0
,x,y,z,imp,location,filtering,group,group_name,gain,offset
id,,,,,,,,,,
1,NaN,NaN,NaN,-1.0,"Pre-Motor Cortex, dorsal",1000Hz,"1 pynwb.ecephys.ElectrodeGroup at 0x5088138896\nFields:\n description: array corresponding to device implanted at PMd\n device: Utah Array(PMd) pynwb.device.Device at 0x5087765776\nFields:\n description: 96 channel utah array\n manufacturer: BlackRock Microsystems\n\n location: Caudal, dorsal Pre-motor cortex, Left hemisphere\n",1,1.0,0.0
2,NaN,NaN,NaN,-1.0,"Pre-Motor Cortex, dorsal",1000Hz,"1 pynwb.ecephys.ElectrodeGroup at 0x5088138896\nFields:\n description: array corresponding to device implanted at PMd\n device: Utah Array(PMd) pynwb.device.Device at 0x5087765776\nFields:\n description: 96 channel utah array\n manufacturer: BlackRock Microsystems\n\n location: Caudal, dorsal Pre-motor cortex, Left hemisphere\n",1,1.0,0.0
3,NaN,NaN,NaN,-1.0,"Pre-Motor Cortex, dorsal",1000Hz,"1 pynwb.ecephys.ElectrodeGroup at 0x5088138896\nFields:\n description: array corresponding to device implanted at PMd\n device: Utah Array(PMd) pynwb.device.Device at 0x5087765776\nFields:\n description: 96 channel utah array\n manufacturer: BlackRock Microsystems\n\n location: Caudal, dorsal Pre-motor cortex, Left hemisphere\n",1,1.0,0.0


In [4]:
trialtable = nwb.trials.to_dataframe()
#trialtable = trialtable[trialtable['correct_reach'] == 0].reset_index()
neurons = nwb.units.to_dataframe()['spike_times'] # Neuron id x Spike Time
time = np.round(nwb.processing['behavior'].data_interfaces['Position'].spatial_series['Hand'].timestamps[:]+ 0.0001,decimals=3)
hand = nwb.processing['behavior'].data_interfaces['Position'].spatial_series['Hand'].data[:] # x, y 
cursor = nwb.processing['behavior'].data_interfaces['Position'].spatial_series['Cursor'].data[:] # x, y
gocue = np.round(trialtable['go_cue_time'], decimals=3)
startmov = np.round(trialtable['move_begins_time'], decimals=3)
endmov = np.round(trialtable['move_ends_time'], decimals=3)
reactiontime = np.round(trialtable['reaction_time'], decimals=3)
target_pos = trialtable['target_positions']
num_neurons = neurons.shape[0]

targeton = np.round(trialtable['target_presentation_time'], decimals=3)

In [5]:
trialtable

,id,start_time,stop_time,target_presentation_time,go_cue_time,reaction_time,move_begins_time,move_ends_time,discard_trial,task_success,...,correct_reach,maze_num_targets,maze_num_barriers,novel_maze,target_positions,frame_details,hit_target_position,target_size,barrier_info,timeseries
0,0,4.707200,8.567133,5.570200,6.517200,0.335,6.819200,7.234200,0,1,...,0,1,2,1,"[[-130, -57]]","[[[-200]], [[200]], [[-155]], [[155]], [[20]]]","[-130, -57]",13,"[[-16.0, 96.0, 59.0, 22.0], [-62.0, -112.0, 43...","[(0, 3226, Eye pynwb.behavior.SpatialSeries at..."
1,1,9.407200,12.112167,10.172200,10.202200,0.305,10.479200,10.789200,0,1,...,0,1,2,1,"[[120, 70]]","[[[-200]], [[200]], [[-155]], [[155]], [[20]]]","[120, 70]",13,"[[-49.0, -59.0, 22.0, 53.0], [-33.0, -100.0, 5...","[(3226, 2081, Eye pynwb.behavior.SpatialSeries..."
2,14,76.347467,80.742333,77.154467,78.135467,0.325,79.189467,79.434467,0,1,...,0,3,9,0,"[[-109, 2], [2, 82], [132, -65]]","[[[-200]], [[200]], [[-155]], [[155]], [[20]]]","[2, 82]",13,"[[-9.0, 52.0, 8.0, 43.0], [-50.0, 91.0, 64.0, ...","[(39894, 3786, Eye pynwb.behavior.SpatialSerie..."
3,21,112.507800,117.207367,113.372800,114.336800,0.295,115.544800,115.879800,0,1,...,0,3,7,0,"[[111, 85], [133, -80], [-116, -5]]","[[[-200]], [[200]], [[-155]], [[155]], [[20]]]","[133, -80]",13,"[[-69.0, -16.0, 69.0, 13.0], [-120.0, -62.0, 1...","[(60541, 4071, Eye pynwb.behavior.SpatialSerie..."
4,26,134.787733,137.847867,135.599733,135.729733,0.365,136.064733,136.444733,0,1,...,0,1,7,0,"[[-105, -76]]","[[[-200]], [[200]], [[-155]], [[155]], [[20]]]","[-105, -76]",13,"[[66.0, -43.0, 9.0, 30.0], [-66.0, 1.0, 70.0, ...","[(75301, 2356, Eye pynwb.behavior.SpatialSerie..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
335,2627,15555.210725,15558.655359,15556.020725,15556.801725,0.290,15557.062725,15557.332725,0,1,...,0,1,0,0,"[[118, 72]]","[[[-200]], [[200]], [[-155]], [[155]], [[20]]]","[118, 72]",13,[],"[(7978837, 2821, Eye pynwb.behavior.SpatialSer..."
336,2629,15564.170725,15568.215359,15564.874725,15565.338725,0.320,15566.657725,15566.922725,0,1,...,0,1,7,0,"[[-116, -5]]","[[[-200]], [[200]], [[-155]], [[155]], [[20]]]","[-116, -5]",13,"[[-69.0, -16.0, 69.0, 13.0], [-120.0, -62.0, 1...","[(7984344, 3451, Eye pynwb.behavior.SpatialSer..."
337,2630,15577.975625,15580.560692,15578.648625,15579.195625,0.325,15579.487625,NaN,0,0,...,0,3,8,0,"[[132, 56], [131, -55], [-118, -7]]","[[[-200]], [[200]], [[-155]], [[155]], [[20]]]","[-118, -7]",13,"[[-64.0, -47.0, 113.0, 7.0], [-64.0, -39.0, 40...","[(7987795, 2581, Eye pynwb.behavior.SpatialSer..."
338,2632,15591.780559,15594.615592,15592.688559,15593.202559,0.360,15593.532559,NaN,0,0,...,0,1,8,0,"[[132, 56]]","[[[-200]], [[200]], [[-155]], [[155]], [[20]]]","[132, 56]",13,"[[-64.0, -47.0, 113.0, 7.0], [-64.0, -39.0, 40...","[(7993352, 2831, Eye pynwb.behavior.SpatialSer..."


In [6]:
num_trials1=0
num_trials2=len(trialtable)

In [7]:
data_list = {'firing_rates': [],
             'events': [],
             'hand': [],
             'cursor': [],
             'result': [],
             'correct_reach': [],
             'maze_num_targets': [],
             'maze_num_barriers': [],
             'target_positions':[],
             'targeton': [],
             'hit_target_pos': [],
             'maze_conds': []}

for trial_id in tqdm(range(num_trials1, num_trials2)):

    min_T = np.round(trialtable['start_time'][trial_id], decimals=3) # Trial start time
    max_T = np.round(trialtable['stop_time'][trial_id], decimals=3) # Trial end time

    mi = np.nonzero(time>min_T)[0][0]
    ma = np.nonzero(time<max_T)[0][-1]

    grid = np.arange(min_T, max_T + 0.001, 0.001) #seconds
    grid = np.round(grid, decimals=3)
    grids = grid[:-1]
    gride = grid[1:]
    num_bins = len(grid)

    neurons_binned = np.zeros((num_bins, num_neurons))
    hand_binned = np.zeros((num_bins, 2))
    cursor_binned = np.zeros((num_bins, 2))
    targets_binned = np.zeros((num_bins,))
    events_binned = np.zeros((num_bins,))
    result= np.zeros((num_bins,))
    correct_reach = np.zeros((num_bins,))
    maze_barriers = np.zeros((num_bins,))
    maze_targets = np.zeros((num_bins,))
    reaction_t = np.zeros((num_bins,))
    targets_pos = np.zeros((num_bins, 2))
    maze_conds = np.zeros((num_bins,))
    hit_target_pos = np.zeros((num_bins, 2))

    #targets_dir += target_dir_degree #target dir

    _, hand_binned[:,0] = np.histogram(hand[mi:ma,0], bins=num_bins-1)
    _, hand_binned[:,1] = np.histogram(hand[mi:ma,1], bins=num_bins-1)

    _, cursor_binned[:,1] = np.histogram(cursor[mi:ma,1], bins=num_bins-1)
    _, cursor_binned[:,1] = np.histogram(cursor[mi:ma,1], bins=num_bins-1)

    #hand_binned = hand[mi:ma]
    #cursor_binned = cursor[mi:ma]
    maze_barriers  += trialtable['maze_num_barriers'][trial_id]
    targets_pos += target_pos[trial_id][0]
    reaction_t += reactiontime[trial_id]

    maze_targets += trialtable['maze_num_targets'][trial_id]

    hit_target_pos += trialtable['hit_target_position'][trial_id]

    result += trialtable['task_success'][trial_id]
    
    correct_reach
    
    maze_conds += trialtable['maze_condition'][trial_id]

    for n in range(num_neurons):
        # Rounding the spiketimes to two decimal places
        rounded_spike_times = np.round(neurons[n], decimals=3)

        neurons_binned[:, n] = np.isin(grid, rounded_spike_times).astype(int)

    if gocue[trial_id] > 0 and gocue[trial_id] <= grid[-1]:

        idx_go = np.where(grid == gocue[trial_id])[0][0]
        events_binned[idx_go] = 2 #go cue

    if startmov[trial_id] > 0:

        idx_startmov = np.where(grid == startmov[trial_id])[0][0]
        events_binned[idx_startmov] = 3 #move start

    if endmov[trial_id] > 0:
        idx_endmov = np.where(grid == endmov[trial_id])[0][0]
        events_binned[idx_endmov] = 4 #move end

    if targeton[trial_id] > 0:
        idx_ton = np.where(grid == targeton[trial_id])[0][0]
        events_binned[idx_ton] = 1 #target on

    data_list['firing_rates'].append(neurons_binned)
    data_list['cursor'].append(cursor_binned)
    data_list['hand'].append(hand_binned)
    data_list['events'].append(events_binned)
    data_list['result'].append(result)
    data_list['maze_num_barriers'].append(maze_barriers)
    data_list['maze_num_targets'].append(maze_targets)
    data_list['target_positions'].append(targets_pos)

100%|██████████| 340/340 [14:46<00:00,  2.61s/it]


In [8]:
data_list.keys()

dict_keys(['firing_rates', 'events', 'hand', 'cursor', 'result', 'maze_num_targets', 'maze_num_barriers', 'target_positions', 'targeton', 'hit_target_pos', 'maze_conds'])

In [9]:
fr = pd.DataFrame(np.vstack(data_list["firing_rates"]), columns=["Neuron"+str(i) for i in range(1, num_neurons+1)])
fr['trial_id'] = np.repeat(range(num_trials1, num_trials2), [arr.shape[0] for arr in data_list['firing_rates']])

In [10]:
fr

,Neuron1,Neuron2,Neuron3,Neuron4,Neuron5,Neuron6,Neuron7,Neuron8,Neuron9,Neuron10,...,Neuron184,Neuron185,Neuron186,Neuron187,Neuron188,Neuron189,Neuron190,Neuron191,Neuron192,trial_id
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
2,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1353144,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,339
1353145,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,339
1353146,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,339
1353147,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,339


In [11]:
h = pd.DataFrame(np.vstack(data_list["hand"]),columns = ["hand_pos_x", "hand_pos_y"])
c = pd.DataFrame(np.vstack(data_list["cursor"]),columns = ["cursor_pos_x", "cursor_pos_y"])
tpos = pd.DataFrame(np.vstack(data_list["target_positions"]),columns = ["target_pos_x", "target_pos_y"])

In [12]:
tpos

,target_pos_x,target_pos_y
0,-130.0,-57.0
1,-130.0,-57.0
2,-130.0,-57.0
3,-130.0,-57.0
4,-130.0,-57.0
...,...,...
1353144,-116.0,-5.0
1353145,-116.0,-5.0
1353146,-116.0,-5.0
1353147,-116.0,-5.0


In [13]:
c

,cursor_pos_x,cursor_pos_y
0,0.0,-62.300000
1,0.0,-62.281788
2,0.0,-62.263575
3,0.0,-62.245363
4,0.0,-62.227150
...,...,...
1353144,0.0,1.831609
1353145,0.0,1.833631
1353146,0.0,1.835652
1353147,0.0,1.837674


In [14]:
h

,hand_pos_x,hand_pos_y
0,-123.012622,-97.224175
1,-122.981497,-97.205969
2,-122.950371,-97.187763
3,-122.919246,-97.169557
4,-122.888121,-97.151351
...,...,...
1353144,0.519484,-33.717984
1353145,0.543974,-33.716079
1353146,0.568465,-33.714174
1353147,0.592955,-33.712270


In [15]:
numtargets = pd.DataFrame(np.hstack(data_list["maze_num_targets"]), columns=['maze_num_target'])
numb = pd.DataFrame(np.hstack(data_list["maze_num_barriers"]), columns=['maze_num_barriers'])

In [16]:
numtargets

,maze_num_target
0,1.0
1,1.0
2,1.0
3,1.0
4,1.0
...,...
1353144,1.0
1353145,1.0
1353146,1.0
1353147,1.0


In [17]:
numtargets.value_counts()

maze_num_target
1.0                788497
3.0                564652
Name: count, dtype: int64

In [18]:
numb

,maze_num_barriers
0,2.0
1,2.0
2,2.0
3,2.0
4,2.0
...,...
1353144,7.0
1353145,7.0
1353146,7.0
1353147,7.0


In [19]:
event = pd.DataFrame(np.hstack(data_list["events"]), columns=['events'])

In [20]:
event

,events
0,0.0
1,0.0
2,0.0
3,0.0
4,0.0
...,...
1353144,0.0
1353145,0.0
1353146,0.0
1353147,0.0


In [21]:
event.value_counts()

events
0.0       1351842
1.0           340
2.0           340
3.0           340
4.0           287
Name: count, dtype: int64

In [22]:
r = pd.DataFrame(np.hstack(data_list["result"]), columns=['result'])

In [23]:
r

,result
0,1.0
1,1.0
2,1.0
3,1.0
4,1.0
...,...
1353144,1.0
1353145,1.0
1353146,1.0
1353147,1.0


In [24]:
ev = pd.concat([h,c,tpos,numtargets,numb,event,r], axis=1)

In [25]:
result = pd.concat([fr,ev], axis=1)

In [26]:
result

,Neuron1,Neuron2,Neuron3,Neuron4,Neuron5,Neuron6,Neuron7,Neuron8,Neuron9,Neuron10,...,hand_pos_x,hand_pos_y,cursor_pos_x,cursor_pos_y,target_pos_x,target_pos_y,maze_num_target,maze_num_barriers,events,result
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-123.012622,-97.224175,0.0,-62.300000,-130.0,-57.0,1.0,2.0,0.0,1.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-122.981497,-97.205969,0.0,-62.281788,-130.0,-57.0,1.0,2.0,0.0,1.0
2,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-122.950371,-97.187763,0.0,-62.263575,-130.0,-57.0,1.0,2.0,0.0,1.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-122.919246,-97.169557,0.0,-62.245363,-130.0,-57.0,1.0,2.0,0.0,1.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-122.888121,-97.151351,0.0,-62.227150,-130.0,-57.0,1.0,2.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1353144,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.519484,-33.717984,0.0,1.831609,-116.0,-5.0,1.0,7.0,0.0,1.0
1353145,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.543974,-33.716079,0.0,1.833631,-116.0,-5.0,1.0,7.0,0.0,1.0
1353146,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.568465,-33.714174,0.0,1.835652,-116.0,-5.0,1.0,7.0,0.0,1.0
1353147,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.592955,-33.712270,0.0,1.837674,-116.0,-5.0,1.0,7.0,0.0,1.0


In [27]:
fr['Neuron1'].describe()

count    1.353149e+06
mean     6.082109e-03
std      7.775038e-02
min      0.000000e+00
25%      0.000000e+00
50%      0.000000e+00
75%      0.000000e+00
max      1.000000e+00
Name: Neuron1, dtype: float64

In [28]:
event['events'].value_counts()

events
0.0    1351842
1.0        340
2.0        340
3.0        340
4.0        287
Name: count, dtype: int64

In [29]:
result

,Neuron1,Neuron2,Neuron3,Neuron4,Neuron5,Neuron6,Neuron7,Neuron8,Neuron9,Neuron10,...,hand_pos_x,hand_pos_y,cursor_pos_x,cursor_pos_y,target_pos_x,target_pos_y,maze_num_target,maze_num_barriers,events,result
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-123.012622,-97.224175,0.0,-62.300000,-130.0,-57.0,1.0,2.0,0.0,1.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-122.981497,-97.205969,0.0,-62.281788,-130.0,-57.0,1.0,2.0,0.0,1.0
2,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-122.950371,-97.187763,0.0,-62.263575,-130.0,-57.0,1.0,2.0,0.0,1.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-122.919246,-97.169557,0.0,-62.245363,-130.0,-57.0,1.0,2.0,0.0,1.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-122.888121,-97.151351,0.0,-62.227150,-130.0,-57.0,1.0,2.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1353144,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.519484,-33.717984,0.0,1.831609,-116.0,-5.0,1.0,7.0,0.0,1.0
1353145,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.543974,-33.716079,0.0,1.833631,-116.0,-5.0,1.0,7.0,0.0,1.0
1353146,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.568465,-33.714174,0.0,1.835652,-116.0,-5.0,1.0,7.0,0.0,1.0
1353147,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.592955,-33.712270,0.0,1.837674,-116.0,-5.0,1.0,7.0,0.0,1.0


In [30]:
# Save the DataFrame to a Parquet file
parquet_file_path = 'data/000070/sub-Jenkins/mc_2_1_errortrials.parquet' 
table = pa.Table.from_pandas(result)
pq.write_table(table, parquet_file_path)